In [4]:
import os
os.environ["LD_LIBRARY_PATH"] = "/home/florian/src/ForschungFlorian/.venv2/lib:/home/florian/src/ForschungFlorian/.venv2/lib/python3.9/site-packages/ccdc/_lib"
os.environ["PYTHONPATH"] = "/home/florian/src/ForschungFlorian/.venv2/bin" 

os.environ["CCDC_TOOLKIT_ASER_DATABASE"] = "/home/florian/CCDC/ccdc-data/csd/as545be"
os.environ["CCDC_TOOLKIT_SQLITE_DATABASE"] = "/home/florian/CCDC/ccdc-data/csd/as545be_ASER.sqlite"
os.environ["CCDC_ISOSTAR_DATA_DIRECTORY"] = "/home/florian/CCDC/ccdc-data/isostar"
os.environ["CCDC_MOGUL_DATA"] = "/home/florian/CCDC/ccdc-data/mogul"
os.environ["CCDC_MOGUL_INITIAflorianTION_FILE"] = "/home/florian/src/ForschungFlorian/.venv2/lib/python3.9/site-packages/ccdc/parameter_files/mogul.ini"
os.environ["CCDC_CROSSMINER_DATABASE"] = "/home/florian/CCDC/ccdc-data/crossminer/csd_pdb_crossminer.feat"
os.environ["CCDC_CROSSMINER_FEATURE_DEFINITIONS"] = "/home/florian/CCDC/ccdc-data/crossminer/feature_definitions"
os.environ["GOLD_DIR"] = "/home/florian/CCDC/ccdc-software/gold/GOLD"

In [5]:
from rdkit import Chem
from rdkit.Chem import rdchem
from rdkit.Chem.rdDetermineBonds import DetermineBonds, DetermineConnectivity, DetermineBondOrders
from rdkit.Chem import Draw
import matplotlib
from ccdc import io
from ccdc.diagram import DiagramGenerator
from ccdc.search import SubstructureSearch, ConnserSubstructure, SMARTSSubstructure, Search
from ccdc.descriptors import CrystalDescriptors
from ccdc.utilities import Histogram
import matplotlib.pyplot as plt
from ccdc.io import MoleculeReader
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import rdSubstructLibrary
from rdkit.Chem import rdmolops
from rdkit.Chem import rdMolTransforms
import os
import numpy as np

In [3]:
ketone_substructure = SMARTSSubstructure("*")

settings = SubstructureSearch.Settings()
settings.no_metals = False
settings.has_3d_coordinates = True
settings.no_disorder = True
settings.max_hits_per_structure = 1
settings.no_errors = True
settings.not_polymeric = True
# settings.must_not_have_elements = ["Sn", "Sb", "Te"]
# settings.must_have_elements = ["H"]
settings.match_enantiomers = "ALWAYS"
settings.no_ions = True

substructure_search = SubstructureSearch(settings)
sub_id = substructure_search.add_substructure(ketone_substructure)
hits = substructure_search.search()
len(hits)

674562

In [6]:
from xyzTomol import read_xyz_all_conformers, parse_filename, build_bonds_and_charges, build_mol_with_coords
import os

folder_path = "data"
mols = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".xyz"):
            xyz_file = os.path.join(root, file)
            zentralatom, oxzahl, liganden = parse_filename(file)
            conformers = read_xyz_all_conformers(xyz_file)
            for atoms, coords in conformers:
                bonds, charges, total_charge = build_bonds_and_charges(atoms, zentralatom, liganden, oxzahl)
                mol = build_mol_with_coords(atoms, coords, bonds, charges)
                mols.append(mol)

# Jetzt kannst du direkt mit mols weiterarbeiten, z.B.:
#for i, mol in enumerate(mols):
    # Beispiel: Ausgabe des Namens oder SMILES
#    print(Chem.MolToSmiles(mol))
#    Chem.MolToMolFile(mol, "/home/florian/CSD_Test_2/data/xyz2.mol", i)

In [7]:
"""
#Changes xyz to rdkit to use data
def ccdc2rdkit(mol):
    pass

folder_path = "data3"
files = os.listdir(folder_path)
mols = []

for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        with open(file_path, "r") as f:
            numAtoms = int(f.readline())
            f.seek(0)
            lines = f.readlines()
            i = 0
            while True:
                mol = lines[(numAtoms+2)*i:(numAtoms+2)*(i+1)]
                if not len(mol):
                    break
                mols.append(Chem.MolFromXYZBlock("".join(mol[:])))
                i += 1
                print(mols)
#[DetermineConnectivity(mol,useVdw=False,charge=-1) for mol in mols];
[DetermineBonds(mol, charge=0) for mol in mols];
for mol in mols:
    Chem.MolToMolFile(mol, "data3/rdkit.mol")
"""

'\n#Changes xyz to rdkit to use data\ndef ccdc2rdkit(mol):\n    pass\n\nfolder_path = "data3"\nfiles = os.listdir(folder_path)\nmols = []\n\nfor root, dirs, files in os.walk(folder_path):\n    for file in files:\n        file_path = os.path.join(root, file)\n        with open(file_path, "r") as f:\n            numAtoms = int(f.readline())\n            f.seek(0)\n            lines = f.readlines()\n            i = 0\n            while True:\n                mol = lines[(numAtoms+2)*i:(numAtoms+2)*(i+1)]\n                if not len(mol):\n                    break\n                mols.append(Chem.MolFromXYZBlock("".join(mol[:])))\n                i += 1\n                print(mols)\n#[DetermineConnectivity(mol,useVdw=False,charge=-1) for mol in mols];\n[DetermineBonds(mol, charge=0) for mol in mols];\nfor mol in mols:\n    Chem.MolToMolFile(mol, "data3/rdkit.mol")\n'

In [8]:
struct2torsion = {}
n_Bonds = 3
for mol in mols:
    paths = Chem.FindAllPathsOfLengthN(mol, n_Bonds, useHs=True)
    for path in paths:
        sub_mol = Chem.PathToSubmol(mol, path)
        path_smiles = Chem.MolToSmiles(sub_mol, kekuleSmiles=False, canonical = True)
        if path_smiles not in struct2torsion:
            struct2torsion[path_smiles] = []
        if len(sub_mol.GetAtoms()) != n_Bonds+1:
            continue
        torsion = rdMolTransforms.GetDihedralDeg(sub_mol.GetConformer(), *range(n_Bonds+1))
        struct2torsion[path_smiles].append(torsion)

In [9]:
#shows all keys from CCDC
struct2torsion.keys()

dict_keys(['[H][PH][V+4][Cl-]', '[H][O][V+4][Cl-]', '[H][CH-][V+4][Cl-]', '[H][PH][V+4][Cl]', '[H][O][V+4][Cl]', '[H][CH-][V+4][Cl]', '[H][PH][V+4][PH2]', '[H][O][V+4][PH2]', '[H][CH-][V+4][PH2]', '[H][CH-][V+4][OH]', '[H][O][V+4][CH2-]', '[H][PH][V+4][OH]', '[H][PH][V+4][CH2-]', '[H][S][Mn+2][Cl-]', '[H][O][Mn+2][Cl-]', '[H][CH-][Mn+2][Cl-]', '[H][CH2][Mn+2][Cl-]', '[H][S][Mn+2][Cl]', '[H][O][Mn+2][Cl]', '[H][CH-][Mn+2][Cl]', '[H][CH2][Mn+2][Cl]', '[H][O][Mn+2][SH]', '[H][CH-][Mn+2][SH]', '[H][CH2][Mn+2][SH]', '[H][CH-][Mn+2][OH]', '[H][CH-][Mn+2][CH3]', '[H][S][Mn+2][OH]', '[H][CH2][Mn+2][OH]', '[H][CH2][Mn+2][CH2-]', '[H][S][Mn+2][CH2-]', '[H][O][Mn+2][CH2-]', '[H][O][Mn+2][CH3]', '[H][S][Mn+2][CH3]', 'O[CH2][Mo+2][Cl-]', '[H][NH][Mo+2][Cl-]', '[H][PH][Mo+2][Cl-]', '[CH3][Mo+2][CH2]O', '[NH2][Mo+2][CH2]O', 'O[CH2][Mo+2][PH2]', '[H][NH][Mo+2][CH3]', '[H][PH][Mo+2][CH3]', '[H][PH][Mo+2][NH2]', '[H][NH][Mo+2][PH2]', '[CH2]1[O][Mo+3]1', '[H][S][Mo+3][OH]', '[H][O][Mo+3][OH]', 'O[CH2][Mo

In [10]:
import json
with open('GOAT_torsion.json', 'w') as f:
     json.dump(struct2torsion, f)

In [11]:
struct2angle = {}
n_Bonds = 2
for mol in mols:
    paths = Chem.FindAllPathsOfLengthN(mol, n_Bonds, useHs=True)
    for path in paths:
        sub_mol = Chem.PathToSubmol(mol, path)
        path_smiles = Chem.MolToSmiles(sub_mol, kekuleSmiles=False, canonical = True)
        if path_smiles not in struct2angle:
            struct2angle[path_smiles] = []
        if len(sub_mol.GetAtoms()) != n_Bonds+1:
            continue
        angle = rdMolTransforms.GetAngleDeg(sub_mol.GetConformer(), *range(n_Bonds+1))
        struct2angle[path_smiles].append(angle)

In [12]:
#shows all keys from CCDC
struct2angle.keys()

dict_keys(['[H][PH][V+4]', '[H][O][V+4]', '[H][CH-][V+4]', '[Cl][V+4][Cl-]', '[PH2][V+4][Cl-]', '[OH][V+4][Cl-]', '[CH2-][V+4][Cl-]', '[PH2][V+4][Cl]', '[OH][V+4][Cl]', '[CH2-][V+4][Cl]', '[PH2][V+4][PH2]', '[OH][V+4][PH2]', '[CH2-][V+4][PH2]', '[H][CH-][H]', '[H]O[H]', '[H]P[H]', '[CH2-][V+4][OH]', '[H][S][Mn+2]', '[H][O][Mn+2]', '[H][CH-][Mn+2]', '[H][CH2][Mn+2]', '[Cl][Mn+2][Cl-]', '[SH][Mn+2][Cl-]', '[OH][Mn+2][Cl-]', '[CH2-][Mn+2][Cl-]', '[CH3][Mn+2][Cl-]', '[SH][Mn+2][Cl]', '[OH][Mn+2][Cl]', '[CH2-][Mn+2][Cl]', '[CH3][Mn+2][Cl]', '[OH][Mn+2][SH]', '[CH2-][Mn+2][SH]', '[CH3][Mn+2][SH]', '[CH2-][Mn+2][OH]', '[CH3][Mn+2][OH]', '[H]C[H]', '[CH2-][Mn+2][CH3]', '[H]S[H]', 'O[CH2][Mo+2]', '[H][NH][Mo+2]', '[H][PH][Mo+2]', '[CH3][Mo+2][Cl-]', '[NH2][Mo+2][Cl-]', '[PH2][Mo+2][Cl-]', '[CH3][Mo+2][CH3]', '[CH3][Mo+2][NH2]', '[CH3][Mo+2][PH2]', '[NH2][Mo+2][PH2]', '[H]N[H]', 'C[O][Mo+3]', 'O[CH2][Mo+3]', '[H][S][Mo+3]', '[H][O][Mo+3]', '[CH3][Mo+3][OH]', '[OH][Mo+3][SH]', '[OH][Mo+3][OH]', '

In [13]:
import json
with open('GOAT_angles.json', 'w') as f:
     json.dump(struct2angle, f)

In [14]:
struct2bond = {}
n_Bonds = 1
for mol in mols:
    paths = Chem.FindAllPathsOfLengthN(mol, n_Bonds, useHs=True)
    for path in paths:
        sub_mol = Chem.PathToSubmol(mol, path)
        path_smiles = Chem.MolToSmiles(sub_mol, kekuleSmiles=False, canonical = True)
        if path_smiles not in struct2bond:
            struct2bond[path_smiles] = []
        if len(sub_mol.GetAtoms()) != n_Bonds+1:
            continue
        bond = rdMolTransforms.GetBondLength(sub_mol.GetConformer(), *range(n_Bonds+1))
        struct2bond[path_smiles].append(bond)

In [15]:
#shows all keys from CCDC
struct2bond.keys()

dict_keys(['[Cl-][V+4]', '[Cl][V+4]', '[PH2][V+4]', '[OH][V+4]', '[CH2-][V+4]', '[H]P', '[H][CH2-]', '[H]O', '[Cl-][Mn+2]', '[Cl][Mn+2]', '[SH][Mn+2]', '[OH][Mn+2]', '[CH2-][Mn+2]', '[CH3][Mn+2]', '[H]S', '[H]C', '[Cl-][Mo+2]', '[CH3][Mo+2]', '[NH2][Mo+2]', '[PH2][Mo+2]', 'CO', '[H]N', '[OH][Mo+3]', '[CH3][Mo+3]', '[SH][Mo+3]', '[NH2][Mo+4]', '[PH2][Mo+4]', '[Cl-][Tc+2]', '[Cl][Tc+2]', '[PH2][Tc+2]', '[OH][Tc+2]', '[Cl-][V+3]', '[Cl][V+3]', '[PH2][V+3]', '[OH][V+3]', '[Cl-][Ti+3]', '[Cl][Ti+3]', '[SH][Ti+3]', '[CH2-][Cr+3]', '[NH2][Cr+3]', '[CH3][Cr+3]', '[OH][Sc+3]', '[CH3][Sc+3]', '[SH][Sc+3]', '[NH2][Sc+3]', '[PH2][Sc+3]', '[SH][Cr+3]', '[Cl-][W+2]', '[Cl][W+2]', '[SH][W+2]', '[NH2][W+2]', '[Cl-][Re+5]', '[SH][Re+5]', '[NH2][Re+5]', '[PH2][Re+5]', '[OH][Re+5]', '[CH2-][Re+5]', '[Cl-][Nb+3]', '[Cl][Nb+3]', '[OH][Nb+3]', '[CH3][Nb+3]', '[PH2][Nb+3]', '[NH2][W+4]', '[PH2][W+4]', '[OH][W+4]', '[CH2-][W+4]', '[Cl-][Re+3]', '[Cl][Re+3]', '[CH2-][Re+3]', '[NH2][Re+3]', '[CH3][Re+3]', '[SH]

In [16]:
import json
with open('GOAT_bonds.json', 'w') as f:
     json.dump(struct2bond, f)

In [17]:
import os
import json

# Mapping wie in Sort.py (kannst du anpassen)
config_index = {
    "OC": "Oktaedrisch",
    "SP": "Quadratisch_Planar",
    "SPY": "Quadratisch_Pyramidal",
    "TS": "T_Foermig",
    "T": "Tetraedrisch",
    "TBPY": "Trigonal_Bipyramidal",
    "TP": "Trigonal_Planar",
    "TPR": "Trigonal_Prismatisch",
    "TPY": "Trigonal_Pyramidal"
}

# Hilfsfunktion: Geometrie aus Dateinamen extrahieren (anpassen falls nötig)
def get_geometry_from_filename(filename):
    for key in config_index:
        if filename.startswith(key + "_"):
            return config_index[key]
    return "Unknown"

folder_path = "data"
geometry_mols = {geo: [] for geo in config_index.values()}
geometry_counts = {geo: 0 for geo in config_index.values()}

for root, dirs, files in os.walk(folder_path):
    for file in files:
        if file.endswith(".xyz"):
            geo = get_geometry_from_filename(file)
            if geo == "Unknown":
                continue
            xyz_file = os.path.join(root, file)
            zentralatom, oxzahl, liganden = parse_filename(file)
            conformers = read_xyz_all_conformers(xyz_file)
            for atoms, coords in conformers:
                bonds, charges, total_charge = build_bonds_and_charges(atoms, zentralatom, liganden, oxzahl)
                mol = build_mol_with_coords(atoms, coords, bonds, charges)
                geometry_mols[geo].append(mol)
            geometry_counts[geo] += 1

# Jetzt für jede Geometrie die drei JSON-Dateien erzeugen
for geo, mols in geometry_mols.items():
    if not mols:
        continue
    struct2torsion = {}
    n_Bonds = 3
    print(f"Processing geometry: {geo} with {len(mols)} molecules")
    for mol in mols:
        paths = Chem.FindAllPathsOfLengthN(mol, n_Bonds, useHs=True)
        for path in paths:
            sub_mol = Chem.PathToSubmol(mol, path)
            path_smiles = Chem.MolToSmiles(sub_mol, kekuleSmiles=False, canonical=True)
            if path_smiles not in struct2torsion:
                struct2torsion[path_smiles] = []
            if len(sub_mol.GetAtoms()) != n_Bonds+1:
                continue
            torsion = rdMolTransforms.GetDihedralDeg(sub_mol.GetConformer(), *range(n_Bonds+1))
            struct2torsion[path_smiles].append(torsion)
    print(f"Processed {len(struct2torsion)} torsion structures for geometry {geo}")
    struct2angle = {}
    n_Bonds = 2
    for mol in mols:
        paths = Chem.FindAllPathsOfLengthN(mol, n_Bonds, useHs=True)
        for path in paths:
            sub_mol = Chem.PathToSubmol(mol, path)
            path_smiles = Chem.MolToSmiles(sub_mol, kekuleSmiles=False, canonical=True)
            if path_smiles not in struct2angle:
                struct2angle[path_smiles] = []
            if len(sub_mol.GetAtoms()) != n_Bonds+1:
                continue
            angle = rdMolTransforms.GetAngleDeg(sub_mol.GetConformer(), *range(n_Bonds+1))
            struct2angle[path_smiles].append(angle)
    print(f"Processed {len(struct2angle)} angle structures for geometry {geo}")
    struct2bond = {}
    n_Bonds = 1
    for mol in mols:
        paths = Chem.FindAllPathsOfLengthN(mol, n_Bonds, useHs=True)
        for path in paths:
            sub_mol = Chem.PathToSubmol(mol, path)
            path_smiles = Chem.MolToSmiles(sub_mol, kekuleSmiles=False, canonical=True)
            if path_smiles not in struct2bond:
                struct2bond[path_smiles] = []
            if len(sub_mol.GetAtoms()) != n_Bonds+1:
                continue
            bond = rdMolTransforms.GetBondLength(sub_mol.GetConformer(), *range(n_Bonds+1))
            struct2bond[path_smiles].append(bond)
    print(f"Processed {len(struct2bond)} bond structures for geometry {geo}")
    # Schreibe die JSON-Dateien direkt ins Hauptverzeichnis, z.B. "GOATbond_Tetraedrisch.json"
    with open(f'Geometry_Data/GOAT_torsion_{geo}.json', 'w') as f:
        json.dump(struct2torsion, f)
    with open(f'Geometry_Data/GOAT_angles_{geo}.json', 'w') as f:
        json.dump(struct2angle, f)
    with open(f'Geometry_Data/GOAT_bonds_{geo}.json', 'w') as f:
        json.dump(struct2bond, f)
    print(f"Saved JSON files for geometry {geo}")
# Ausgabe wie viele Dateien pro Geometrie verarbeitet wurden
for geo, count in geometry_counts.items():
    print(f"{geo}: {count} Dateien verarbeitet")

Processing geometry: Oktaedrisch with 454430 molecules
Processed 2877 torsion structures for geometry Oktaedrisch
Processed 1896 angle structures for geometry Oktaedrisch
Processed 367 bond structures for geometry Oktaedrisch
Saved JSON files for geometry Oktaedrisch
Processing geometry: Quadratisch_Planar with 9122 molecules
Processed 384 torsion structures for geometry Quadratisch_Planar
Processed 258 angle structures for geometry Quadratisch_Planar
Processed 55 bond structures for geometry Quadratisch_Planar
Saved JSON files for geometry Quadratisch_Planar
Processing geometry: Quadratisch_Pyramidal with 34000 molecules
Processed 447 torsion structures for geometry Quadratisch_Pyramidal
Processed 300 angle structures for geometry Quadratisch_Pyramidal
Processed 63 bond structures for geometry Quadratisch_Pyramidal
Saved JSON files for geometry Quadratisch_Pyramidal
Processing geometry: T_Foermig with 888 molecules
Processed 105 torsion structures for geometry T_Foermig
Processed 84 a